# DGCNN (Dynamic Graph CNN) for learning on Point Clouds

Prepare library and installations

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install ipyvolume

1.11.0+cu113
     |████████████████████████████████| 7.9 MB 31.7 MB/s 
     |████████████████████████████████| 3.5 MB 41.0 MB/s 
     |████████████████████████████████| 2.5 MB 38.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 28.7 MB/s 
     |████████████████████████████████| 3.4 MB 6.4 MB/s 
     |████████████████████████████████| 260 kB 66.3 MB/s 
     |████████████████████████████████| 271 kB 69.5 MB/s 


Setup external visualization tools and import relevant packages

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import os.path as osp
import numpy as np
import ipyvolume as ipv
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.cm as cm
import matplotlib as mpl


import torch
import torch.nn.functional as F
from torch.nn import Linear

import torch_geometric.transforms as T
from torch_geometric.datasets import ModelNet
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_max_pool, MLP
from torch_geometric.nn import knn_graph
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing

Get the dataset.

> Make sure the google drive is mounted

In [ ]:
data_root = "/content/drive/MyDrive/datasets/"

In [ ]:
path = osp.join(data_root, 'ModelNet10')
# pre_transform, transform = T.NormalizeScale(), T.SamplePoints(512)
pre_transform, transform = T.NormalizeScale(), T.SamplePoints(2048)
train_dataset = ModelNet(path, '10', True, transform, pre_transform)
test_dataset = ModelNet(path, '10', False, transform, pre_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,
                          num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         num_workers=1)

## Quick Data exploration

In [ ]:
def visualize_pc(points, color=False):
    norm = mpl.colors.Normalize(vmin=0, vmax=1.5)
    cmap = cm.hsv
    dist = np.sqrt(np.sum((points - points.mean())**2, 1))
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    cols = m.to_rgba(dist)

    ipv.figure()
    if color:
        ipv.scatter(points[:, 0], points[:, 1], points[:, 2], marker='sphere', color=cols)
    else:
        ipv.scatter(points[:, 0], points[:, 1], points[:, 2], marker='sphere', color='blue')
    ipv.show()

In [ ]:
# for ix in range(500, 510):
#     print(ix, train_dataset[ix].y.item())

In [ ]:
sample = train_dataset[3510].pos.data.numpy()
visualize_pc(sample, True)

In [ ]:
pos = train_dataset[510].pos
edge_index = knn_graph(pos, k=3)

In [ ]:
points = pos.data.numpy()
len(edge_index.T)

1536

In [ ]:
# ipv.figure()

# for ix in edge_index.T[:300]:
#     p1 = points[ix[0]]
#     p2 = points[ix[1]]
#     ipv.plot([p1[0], p2[0]], 
#              [p1[1], p2[1]], 
#              [p1[2], p2[2]], color='black')

# ipv.scatter(points[:, 0], points[:, 1], points[:, 2], color='blue', marker='sphere')

# ipv.show()

# Network

In [ ]:
class EdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='max') #  "Max" aggregation.
        self.mlp = Seq(Linear(2 * in_channels, out_channels),
                       ReLU(),
                       Linear(out_channels, out_channels))

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # x_i has shape [E, in_channels]
        # x_j has shape [E, in_channels]

        tmp = torch.cat([x_i, x_j - x_i], dim=1)  # tmp has shape [E, 2 * in_channels]
        return self.mlp(tmp)

In [ ]:
class DynamicEdgeConv(EdgeConv):
    def __init__(self, in_channels, out_channels, k=6):
        super().__init__(in_channels, out_channels)
        self.k = k

    def forward(self, x, batch=None):
        edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow)
        return super().forward(x, edge_index)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, out_channels, k=20, aggr='max'):
        super().__init__()

        self.conv1 = DynamicEdgeConv(3, 64, k)
        self.conv2 = DynamicEdgeConv(64, 128, k)
        self.lin1 = Linear(128 + 64, 1024)

        self.mlp = MLP([1024, 512, 256, out_channels], dropout=0.5,
                       batch_norm=False)

    def forward(self, data):
        pos, batch = data.pos, data.batch
        x1 = self.conv1(pos, batch)
        x2 = self.conv2(x1, batch)
        out = self.lin1(torch.cat([x1, x2], dim=1))
        out = global_max_pool(out, batch)
        out = self.mlp(out)
        return F.log_softmax(out, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(train_dataset.num_classes, k=20).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

In [ ]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
    return total_loss / len(train_dataset)

In [ ]:
def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            pred = model(data).max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)

In [ ]:
for epoch in range(1, 10):
    loss = train()
    test_acc = test(test_loader)
    print(f'Epoch {epoch:03d}, Loss: {loss:.4f}, Test: {test_acc:.4f}')
    scheduler.step()

Epoch 001, Loss: 1.5221, Test: 0.4714
Epoch 002, Loss: 0.7603, Test: 0.7621
Epoch 003, Loss: 0.5100, Test: 0.7401
Epoch 004, Loss: 0.4157, Test: 0.7952
Epoch 005, Loss: 0.3568, Test: 0.7930
Epoch 006, Loss: 0.3381, Test: 0.7863
Epoch 007, Loss: 0.3131, Test: 0.8348
Epoch 008, Loss: 0.2720, Test: 0.8282
Epoch 009, Loss: 0.2675, Test: 0.8822


In [ ]:
# save the trained model
torch.save(model, os.path.join(data_root, "trained_model_v2_10epochs.pth"))